In [ ]:
# Add parent directory to Python path
import sys
from pathlib import Path
sys.path.append(str(Path('../').resolve()))

# Core libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Financial APIs
import yfinance as yf
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.fundamentaldata import FundamentalData

# Technical analysis
import pandas_ta as ta

# Database and utilities
import sqlalchemy
from dotenv import load_dotenv
from src.database import get_database_connection, get_api_key

# AI
from openai import OpenAI

# Load environment
load_dotenv()
print("✅ Environment loaded and libraries imported successfully!")


In [ ]:
def fetch_market_data(symbol, period="1y", start_date=None, end_date=None):
    """
    Fetch OHLCV market data for a given symbol.
    
    Args:
        symbol (str): Stock symbol (e.g., 'INTC')
        period (str): Time period ('1y', '2y', etc.) - used if dates not specified
        start_date (str): Start date in 'YYYY-MM-DD' format
        end_date (str): End date in 'YYYY-MM-DD' format
    
    Returns:
        pandas.DataFrame: OHLCV data with date index
    """
    try:
        ticker = yf.Ticker(symbol)
        
        if start_date and end_date:
            data = ticker.history(start=start_date, end=end_date)
        else:
            data = ticker.history(period=period)
        
        if data.empty:
            print(f"⚠️  No data retrieved for {symbol}")
            return None
            
        # Clean the data
        data = data.drop(columns=['Dividends', 'Stock Splits'], errors='ignore')
        data.index = data.index.date  # Convert to date only
        
        print(f"✅ Retrieved {len(data)} days of data for {symbol}")
        return data
        
    except Exception as e:
        print(f"❌ Error fetching data for {symbol}: {e}")
        return None

# Test the function
print("🔍 Testing market data collection...")
test_data = fetch_market_data("INTC", period="1mo")
if test_data is not None:
    print(f"📅 Date range: {test_data.index[0]} to {test_data.index[-1]}")
    print(f"💰 Latest close: ${test_data['Close'][-1]:.2f}")
    display(test_data.tail(3))
